In [ ]:
! pip install -r drive/MyDrive/Diploma/requirements.txt >& /dev/null

In [ ]:
! nvidia-smi

Fri May 14 11:47:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import torch
from torch import nn

import wandb

from tqdm.notebook import tqdm

from drive.MyDrive.Diploma.environment import KGEnv
from drive.MyDrive.Diploma.utils import LabelSmoothingCrossEntropy, load_config
from drive.MyDrive.Diploma.dataset import KGDataset
from drive.MyDrive.Diploma.train_emb import train_emb, evaluate_emb
from drive.MyDrive.Diploma.beam_search import get_ranks
from drive.MyDrive.Diploma.embed_model import ComplEx
from drive.MyDrive.Diploma.metrics import *

from drive.MyDrive.Diploma.load_config import config

%load_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
env = KGEnv(config['train_triplets_path'])

entities_num = env.entities_num
relations_num = env.relations_num

In [ ]:
train_env = KGEnv(config['train_triplets_path'])
test_env = KGEnv(config['test_triplets_path'], train=False)

emb_model = ComplEx(entities_num, relations_num, hid_dim=256).to(device)
optimizer = torch.optim.Adam(emb_model.parameters(), lr=0.001)
criterion = LabelSmoothingCrossEntropy() 

In [ ]:
wandb.init(project="RL4KGQA", name="Embed model")

In [ ]:
np.random.shuffle(train_env.triplets)
np.random.shuffle(test_env.triplets)

In [ ]:
for epoch in tqdm(range(config['num_epochs'])):
    train_loss, train_accuracy, train_ranks = train_emb(
        emb_model, train_env.triplets, optimizer, criterion, batch_size=128
    )
    test_loss, test_accuracy, test_ranks = evaluate_emb(
        emb_model, test_env.triplets, criterion, batch_size=64
    )

    wandb.log({"Train CE loss": train_loss,
               "Train accuracy": train_accuracy,
               "Test CE loss": test_loss,
               "Test accuracy": test_accuracy,
               "Train MMR": mmr(train_ranks),
               "Test MMR": mmr(test_ranks),
               "Train HIT@10": hit_k(train_ranks, k=10),
               "Test HIT@10": hit_k(test_ranks, k=10)})

In [ ]:
loss, accuracy, ranks = evaluate_emb(
        emb_model, test_env.triplets, criterion, batch_size=256
)

In [ ]:
print('Test  | HIT@1: {:.3},\t HIT@10: {:.3},\t MMR: {:.3}'.format(
    hit_k(ranks, k=1), hit_k(ranks, k=10), mmr(ranks)
))

Test  | HIT@1: 0.219,	 HIT@10: 0.853,	 MMR: 0.411
